In [2]:
import gymnasium as gym

In [10]:
env = gym.make('Blackjack-v1', natural=False, sab=False)#añadir render_mode="human" para q pete

#Métodos:
#reset(self) : Reinicia el estado del entorno, a su estado inicial, devolviendo una observación de dicho estado.
#step(self, action) : "Avanza" un timestep. Devuelve: observation, reward, done, info.
#render(self) : Muestra en pantalla el entorno.
#close(self) : Finaliza con la instancia del agente.
#seed(self) : Establece la semilla aleatoria del generador de números aleatorios del presente entorno.

#Atributos:
#action_space : El objeto de tipo Space correspondiente al espacio de acciones válidas.
#observation_space : El objeto de tipo Space correspondiente a todos los rangos posibles de observaciones.
#reward_range : Tupla que contiene los valores mínimo y máximo de recompensa posible.


In [4]:
print("Tamaño de espacio de estados", env.observation_space)
print("Estado aleatorio", env.observation_space.sample())
size_estados = env.observation_space[0].n * env.observation_space[1].n * env.observation_space[2].n
print("Hay", size_estados, " estados posibles.")
print("Acciones posibles", env.action_space.n)
print("Acción aleatoria", env.action_space.sample())
size_acciones = env.action_space.n
print("Hay", size_acciones, " acciones posibles.")

Tamaño de espacio de estados Tuple(Discrete(32), Discrete(11), Discrete(2))
Estado aleatorio (10, 0, 0)
Hay 704  estados posibles.
Acciones posibles 2
Acción aleatoria 1
Hay 2  acciones posibles.


In [9]:
# Reseteamos el entorno y obtenemos el estado inicial
estado = env.reset()

# Imprime el estado inicial
print("Estado inicial:", estado)

# Simulamos una acción para ver cómo se actualiza el estado
accion = 1  # Por ejemplo, podemos elegir 'hit'
nuevo_estado, recompensa, hecho, info, a = env.step(accion)

print("Nuevo estado:", nuevo_estado)
print("Recompensa:", recompensa)
print("Terminado:", hecho)
print("Info:", info)

Estado inicial: ((16, 2, 0), {})
Nuevo estado: (22, 2, 0)
Recompensa: -1.0
Terminado: True
Info: False
